심층 신경망
- 인공 신경망에 층을 여러개 추가

In [1]:
from tensorflow import keras


In [8]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
from sklearn.model_selection import train_test_split

In [5]:
train_input.shape

(60000, 28, 28)

In [9]:
# 이미지 픽섹 범위로 나누고 스케일하기
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [10]:
# 인공 신경망 모델에 층을 2개 추가
# 입력층과 출력층 사이에 밀집층 추가

# 은닉층(hidden layer) : 입력층과 출력층 사이에 있는 모든 함수 

# 회귀를 위한 신경망의 출력층에는 어떤 활성화 함수 사용?
# >> 회귀는 임이의 어떤 수 이므로 활성화 함수 적용x 
# Dense의 activation 매개변수에 아무런 값도 지정x

In [11]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

# dense1 은닉층 10개의 뉴런을 가진 밀집층
# 은닉층의 뉴런 개수를 정하는 데는 특별한 기준은 없고 , 몇 개의 뉴런을 두어야 할지 판단하기 위해서 상당하 경험 필요로 함.
# 제약사항 : 적어도 출력층의 뉴런보다는 많아야 함.
# dense2 출력층> 10개 클래스를 분류하므로 10개 뉴런  

In [12]:
# 심층 신경망 만들기(deep neural newtwork, DNN)
model = keras.Sequential([dense1, dense2])

# Sequential클래스의 객체를 만들 때 여러 개의 층을 추가하려면 dense1과 dense2를 리스트로 만들어 전달.
# 주의 : 출력층을 가장 마지막에 두어야 함.
# 가장 처음 등장하는 은늑층에서 마지막 출력층 순으로 나열

In [13]:
# summary() 메서드 호출 >> 층에 대한 유용한 정보 얻을 수 있음
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층을 추가하는 다른 방법

In [ ]:
# 앞에서는 Dense 클래스 객체 dense1, dense2를 만들어 Squential 클래스에 전달했는데 
# 이 두 객체를 따로 저장해 쓸일이 없어 
# Sequential 클래스와 생성자 안에서 바로  Dense 클래스의 객체를 만드는 경우가 많음

In [15]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name = 'hidden'),
    keras.layers.Dense(10, activation='softmax', name = 'output')
], name= '패션 MNIST 모델')

# 추가되는 층을 한 눈에 볼 수 있음.

In [16]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [18]:
# add() 메서드 사용하여 Sequential 클래스 층 추가하기

model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               78500     
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5733 - accuracy: 0.8064
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4142 - accuracy: 0.8520
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3784 - accuracy: 0.8640
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3556 - accuracy: 0.8717
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3373 - accuracy: 0.8776


In [ ]:
# 성능이 향상됨을 확인

렐루 함수

In [ ]:
# 초창기엔 인공 신경망의 은닉층에 시그모이드 함수를 많이 사용했는데
# 시그모이드 함수는 오른쪽과 왼쪽 끝으로갈 수록 그래프가 누워있어 올바른 출력을 만드는데 
# 신속하게 대응하지 못함

In [ ]:
# 특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 함.
# 개선을 위해 제안된 함수가 렐루(relu) 함수.

# 렐루 함수 : 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력 통과
        # :음수일 경우 0으로 만듦

In [ ]:
# 렐루 함수 : max(0,z)와 같이 쓸 수 있음.
# 렐루함수는 특히 이미지 처리에서 좋은 성능을 냄.

In [ ]:
# 패녓 MNIST 데이터는28*28 크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의
# reshape() 메서드를 사용해 1차원으로 펼쳤음

# >> keras에 Flatten 층을 제공함

In [22]:
# Flatten함수 : 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할
# 입력에 곱해지는 가중치나 절편 없음.

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 100)               78500     
                                                                 
 dense_6 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [26]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input /225.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [30]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=10)

Epoch 1/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3059 - accuracy: 0.8917
Epoch 2/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2932 - accuracy: 0.8959
Epoch 3/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2876 - accuracy: 0.8992
Epoch 4/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2791 - accuracy: 0.9016
Epoch 5/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2707 - accuracy: 0.9046
Epoch 6/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2655 - accuracy: 0.9075
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2623 - accuracy: 0.9075
Epoch 8/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2579 - accuracy: 0.9110
Epoch 9/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2514 - accuracy: 0.9120
Epoch 10/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.248

In [31]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.4117 - accuracy: 0.8826


[0.4117205739021301, 0.8825833201408386]

옵티마이저

In [ ]:
# 이제까지 다룬 하이퍼 파라미터 
# : 은늑층의 개수, 뉴런 개수, 활성화 함수, 층의 종류, 배치 사이즈 매개변수, 에포크 매개변수 등

# keras는 기본적으로 미니배치 경사 하강법을 사용하여 미니배치 개수 32
# fit() 메서드의 batch_size 매개변수에서 조정 가능 > 하이퍼파라미터
# epochs 매개변수도 하이퍼 파라미터

In [ ]:
옵티마이저(optimizer) : keras에서 제공하는 다양한 종류의 경사 하강법 알고리즘.
compile() 메서드에서 카라스의 기본 경사 